In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [5]:
def train_happy_sad_model():
    DESIRED_ACCURACY = 0.999
    class Mycallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            if(logs.get('acc') > DESIRED_ACCURACY):
                self.model.stop_training = True
    callback = Mycallback()
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16 , (3,3) , activation='relu' , input_shape=(150,150,3)),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(32 , (3,3) , activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512 , activation='relu'),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1/255)
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150,150),
        batch_size=80,
        class_mode='binary'
    )

    history = model.fit(train_generator , steps_per_epoch=8 , epochs=15 , verbose=1 , callbacks=[callback] )
    return history.history['accuracy'][-1]

    

In [ ]:
train_happy_sad_model()